In [11]:
# # All pip installs
# !pip install langchain
# !pip install -qU langchain-groq
# !pip install langchain_community

# Load JSON

In [1]:
import json

with open('apiKeys.json', 'r') as file:


    apiKeys = json.load(file)

In [2]:
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = apiKeys['LANGCHAIN_API_KEY']

# Loading LLAMA Model

In [10]:
import os

os.environ["GROQ_API_KEY"] = apiKeys['GROQ_API_KEY']

from langchain_groq import ChatGroq

model = ChatGroq(model="llama-3.1-70b-versatile", temperature=1.2, max_tokens=5000)

## Example Usage

In [11]:
prompt = f"Hi, What's up!"

response = model.invoke([prompt])
response.content


"Hello! Not much, just here and ready to chat! How about you? How's your day going so far?"

# Workflow of the project

diagram-20240715.svg

# Performing the last part of the workflow

## Problem 1

```
Input  : Concept, Theme
Output : JSON Parsable
```

In [ ]:
from langchain.chains import SimpleSequentialChain
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain_core.messages import HumanMessage, SystemMessage,AIMessage


In [ ]:
concept = "strings"
theme = "animie"

# One Shot Learning

In [ ]:
problem_statement = "Write a python program to add the total scores of the batsman in his two innings"
function_name = "add"
function_template = "def add_scores(score1, score2):\n    # Your code here\n    pass"
test_cases = [
    {
        "input": "10, 20",
        "output": "30"
    },
    {
        "input": "46, 52",
        "output": "98"
    }
]

example_output = f"""
{{
    "problem_statement": "{problem_statement}",
    "function_name": "{function_name}",
    "function_template": "{function_template}",
    "test_cases": [
        {{
            "input": "{test_cases[0]['input']}",
            "output": "{test_cases[0]['output']}"
        }},
        {{
            "input": "{test_cases[1]['input']}",
            "output": "{test_cases[1]['output']}"
        }}
    ]
}}
"""



In [ ]:
messages = [
    SystemMessage(content="Generate a valid python programming question with 2 or more test cases for the given concept and theme in json format strictly."),
    HumanMessage(content=f"Concept : functions, Theme : cricket"),
    AIMessage(content=example_output),
    HumanMessage(content=f"Concept : {concept}, Theme : {theme}")
]


In [ ]:
print(model.invoke(messages).content)

{
    "problem_statement": "Write a python program to extract the name of the anime from a given string",
    "function_name": "anime_name",
    "function_template": "def anime_name(string):
    # Your code here
    pass",
    "test_cases": [
        {
            "input": "The anime 'Attack on Titan' is a popular show",
            "output": "Attack on Titan"
        },
        {
            "input": "The popular anime series is 'Fullmetal Alchemist'",
            "output": "Fullmetal Alchemist"
        }
    ]
}


# A module which makes us questions

In [ ]:
import random
import json

class QuestionMaker:

    def __init__(self, userConcepts, userInterests, model_name="llama3-8b-8192"):

        self.userConcepts = userConcepts
        self.userInterests = userInterests
        self.model = ChatGroq(model=model_name)


    def frame_a_question(self, userConcept, userInterest):
        problem_statement_1 = "Write a python program to add the total scores of the batsman in his two innings"
        function_name_1 = "add"
        function_template_1 = "def add_scores(score1, score2):\\n    # Your code here\\n    pass"
        test_cases_1 = [
            {
                "input": "10, 20",
                "output": "30"
            },
            {
                "input": "46, 52",
                "output": "98"
            }
        ]

        example_output_1 = f"""
        {{
            "userConcept" : "functions",
            "userInterest" : "cricket"
            "problem_statement": "{problem_statement_1}",
            "function_name": "{function_name_1}",
            "function_template": "{function_template_1}",
            "test_cases": [
                {{
                    "input": "{test_cases_1[0]['input']}",
                    "output": "{test_cases_1[0]['output']}"
                }},
                {{
                    "input": "{test_cases_1[1]['input']}",
                    "output": "{test_cases_1[1]['output']}"
                }}
            ]
        }}
        """

        problem_statement_2 = "Write a Python program to read a file named 'fruits.txt' and list out all the fruits in a python list."
        function_name_2 = "display_fruits"
        function_template_2 = "def display_fruits(filename):\\n    # Your code here\\n    pass"
        test_cases_2 = [
            {
                "input": "fruits.txt",
                "content" : "apple \\n mango \\n banana",
                "output": "['apple', 'mango', 'banana']"
            },
            {
                "input": "fruits.txt",
                "content" : "blueberry \\n strawberry \\n raspberry",
                "output": "['blueberry', 'strawberry', 'raspberry']"
            }
        ]

        example_output_2 = f"""
        {{
            "userConcept" : "file handling",
            "userInterest" : "fruits"
            "problem_statement": "{problem_statement_2}",
            "function_name": "{function_name_2}",
            "function_template": "{function_template_2}",
            "test_cases": [
                {{
                    "input": "{test_cases_2[0]['input']}",
                    "content" : "{test_cases_2[0]['content']}",
                    "output": "{test_cases_2[0]['output']}"
                }},
                {{
                    "input": "{test_cases_2[1]['input']}",
                    "content" : "{test_cases_2[1]['content']}",
                    "output": "{test_cases_2[1]['output']}"
                }}
            ]
        }}
        """


        messages = [
            SystemMessage(content=f"""
            Generate a valid python programming question with 2 or more test cases for the given
            concept and theme in json format strictly without any other extra sentances.
            """),
            HumanMessage(content=f"Concept : functions, Theme : cricket"),
            AIMessage(content=example_output_1),
            HumanMessage(content=f"Concept : file handling, Theme : fruits"),
            AIMessage(content=example_output_2),
            HumanMessage(content=f"Concept : {userConcept}, Theme : {userInterest}")
        ]

        return model.invoke(messages).content

    def get_questions(self, n):

        questions = set()

        while len(questions) < n:
            userConcept = random.choice(self.userConcepts)
            userInterest = random.choice(self.userInterests)
            question = self.frame_a_question(userConcept, userInterest)
            questions.add(question)

        return list(questions)







# Having 1 user and many topics and interests.

In [ ]:
user_json = [
    {
        "username" : "example",
        "rollNumber" : "21F1000001",
        "userConcepts" : ["functions", "file handling", "list comprehension"],
        "userInterests" : ["cricket", "tamil movies", "tamil music", "cars", "animie"]
    }
]


In [ ]:
questionMaker = QuestionMaker(user_json[0]["userConcepts"], user_json[0]["userInterests"])

userQuestions = questionMaker.get_questions(10)

print(userQuestions)

['{\n    "userConcept" : "file handling",\n    "userInterest" : "cars"\n    "problem_statement": "Write a Python program to read a file named \'car_details.txt\' and display the name and model of each car.",\n    "function_name": "display_cars",\n    "function_template": "def display_cars(filename):\\n    # Your code here\\n    pass",\n    "test_cases": [\n        {\n            "input": "car_details.txt",\n            "content" : "Toyota Corolla \\n Honda Civic \\n Ford Mustang",\n            "output": "[\'Toyota Corolla\', \'Honda Civic\', \'Ford Mustang\']"\n        },\n        {\n            "input": "car_details.txt",\n            "content" : "Tesla Model 3 \\n Volkswagen Golf \\n Nissan Altima",\n            "output": "[\'Tesla Model 3\', \'Volkswagen Golf\', \'Nissan Altima\']"\n        }\n    ]\n}', '{\n            "userConcept" : "file handling",\n            "userInterest" : "tamil music"\n            "problem_statement": "Write a Python program to read a file named \'tamil_s

In [ ]:

for i in range(len(userQuestions)):
    print(userQuestions[i])

{
    "userConcept" : "file handling",
    "userInterest" : "cars"
    "problem_statement": "Write a Python program to read a file named 'car_details.txt' and display the name and model of each car.",
    "function_name": "display_cars",
    "function_template": "def display_cars(filename):\n    # Your code here\n    pass",
    "test_cases": [
        {
            "input": "car_details.txt",
            "content" : "Toyota Corolla \n Honda Civic \n Ford Mustang",
            "output": "['Toyota Corolla', 'Honda Civic', 'Ford Mustang']"
        },
        {
            "input": "car_details.txt",
            "content" : "Tesla Model 3 \n Volkswagen Golf \n Nissan Altima",
            "output": "['Tesla Model 3', 'Volkswagen Golf', 'Nissan Altima']"
        }
    ]
}
{
            "userConcept" : "file handling",
            "userInterest" : "tamil music"
            "problem_statement": "Write a Python program to read a file named 'tamil_songs.txt' and list out the song names in a pyth

In [ ]:
len(userQuestions)

10

In [ ]:
json.loads(userQuestions[3])

{'userConcept': 'functions',
 'userInterest': 'cars',
 'problem_statement': 'Write a python program to calculate the total cost of a car including taxes and insurance',
 'function_name': 'calculate_total_cost',
 'function_template': 'def calculate_total_cost(price, tax_rate, insurance_rate):\n    # Your code here\n    pass',
 'test_cases': [{'input': '20000, 0.08, 0.05', 'output': '21400.0'},
  {'input': '35000, 0.1, 0.03', 'output': '37345.0'}]}

In [ ]:
from langchain.prompts import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.schema import HumanMessage, AIMessage, SystemMessage

import random
import json

class QuestionMaker:
    def __init__(self, user_concept, user_interest, model_name="llama3-8b-8192"):
        self.user_concept = user_concept
        self.user_interest = user_interest
        self.llm = ChatGroq(model=model_name)

    def create_question_name_chain(self):
        messages = [
            SystemMessage(content="You are an expert at creating concise Python question names."),
            HumanMessage(content="Concept: loops\nInterest: space exploration"),
            AIMessage(content="Mars Rover Path Simulator"),
            HumanMessage(content="Concept: {concept}\nInterest: {interest}")
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        return LLMChain(llm=self.llm, prompt=prompt)

    def create_detailed_description_chain(self):
        messages = [
            SystemMessage(content="You are an expert at creating detailed descriptions for Python coding questions."),
            HumanMessage(content="""
Question Name: "Mars Rover Path Simulator"
Concept: loops
Interest: space exploration
            """),
            AIMessage(content="""
Create a Python program that simulates the path of a Mars rover using loops. The program should allow the user to input a series of movement commands (e.g., forward, left, right) and use a loop to process these commands, updating the rover's position on a 2D grid representing the Martian surface. The program should track the rover's final position and the path it took. This exercise will help practice using loops to iterate through commands and update variables based on conditions.
            """),
            HumanMessage(content="""
Question Name: {question_name}
Concept: {concept}
Interest: {interest}
            """)
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        return LLMChain(llm=self.llm, prompt=prompt)

    def create_sample_code_chain(self):
        messages = [
            SystemMessage(content="You are an expert Python programmer creating function templates for coding questions."),
            HumanMessage(content="""
Question Name: "Mars Rover Path Simulator"
Description: Create a Python program that simulates the path of a Mars rover using loops. The program should allow the user to input a series of movement commands (e.g., forward, left, right) and use a loop to process these commands, updating the rover's position on a 2D grid representing the Martian surface. The program should track the rover's final position and the path it took.
            """),
            AIMessage(content="""
def simulate_mars_rover(commands):
    # Your code here
    pass
            """),
            HumanMessage(content="""
Question Name: {question_name}
Description: {description}
            """)
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        return LLMChain(llm=self.llm, prompt=prompt)

    def create_test_cases_chain(self):
        messages = [
            SystemMessage(content="You are an expert Python programmer creating test cases for coding questions."),
            HumanMessage(content="""
Question Name: "Mars Rover Path Simulator"
Description: Create a Python program that simulates the path of a Mars rover using loops. The program should allow the user to input a series of movement commands (e.g., forward, left, right) and use a loop to process these commands, updating the rover's position on a 2D grid representing the Martian surface. The program should track the rover's final position and the path it took.
Sample code:
def simulate_mars_rover(commands):
    # Your code here
    pass
            """),
            AIMessage(content="""
1. Input: ["forward", "right", "forward", "left", "forward"]
   Expected Output: Final Position: (2, 1), Path: [(0, 0), (0, 1), (1, 1), (2, 1)]

2. Input: ["right", "forward", "left", "forward", "forward"]
   Expected Output: Final Position: (1, 2), Path: [(0, 0), (1, 0), (1, 1), (1, 2)]
            """),
            HumanMessage(content="""
Question Name: {question_name}
Description: {description}
Sample code:
{sample_code}
            """)
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        return LLMChain(llm=self.llm, prompt=prompt)
    def frame_a_question(self):
        name_chain = self.create_question_name_chain()
        description_chain = self.create_detailed_description_chain()
        code_chain = self.create_sample_code_chain()
        test_cases_chain = self.create_test_cases_chain()

        question_name = name_chain.run(concept=self.user_concept, interest=self.user_interest)
        description = description_chain.run(question_name=question_name, concept=self.user_concept, interest=self.user_interest)
        sample_code = code_chain.run(question_name=question_name, description=description)
        test_cases = test_cases_chain.run(question_name=question_name, description=description, sample_code=sample_code)

        # Parse test cases into a structured format
        test_cases_list = self.parse_test_cases(test_cases)

        question_data = {
            "userConcept": self.user_concept,
            "userInterest": self.user_interest,
            "questionName": question_name,
            "problemStatement": description,
            "functionTemplate": sample_code,
            "testCases": test_cases_list
        }

        return json.dumps(question_data, indent=2)

    def parse_test_cases(self, test_cases_text):
        # This is a simplified parser. You might need to adjust it based on the actual output format.
        lines = test_cases_text.strip().split('\n')
        test_cases = []
        for i in range(0, len(lines), 2):
            if i+1 < len(lines):
                test_case = {
                    "input": lines[i].split(': ')[-1],
                    "output": lines[i+1].split(': ')[-1]
                }
                test_cases.append(test_case)
        return test_cases

    def get_questions(self, n):
        questions = set()
        while len(questions) < n:
            question = self.frame_a_question()
            questions.add(question)
        return list(questions)

In [ ]:
questionMaker = QuestionMaker("functions", "ipl")
print(questionMaker.get_questions(1))

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use RunnableSequence, e.g., `prompt | llm` instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:139: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


['{\n  "userConcept": "functions",\n  "userInterest": "ipl",\n  "questionName": "Here is a Python question name that combines the concept of \\"loops\\" with an interest in \\"space exploration\\":\\n\\n\\"Galactic Orbit Simulation: Looping Through Celestial Bodies\\"\\n\\nLet me know if you\'d like me to come up with another one!",\n  "problemStatement": "Here is a detailed description for the question:\\n\\n**Question Name:** \\"Mars Rover Path Simulator\\"\\n**Concept:** Loops\\n**Interest:** Space Exploration\\n\\n**Description:**\\n\\nCreate a Python program that simulates the path of a Mars rover on a 2D grid representing the Martian surface. The program should allow the user to input a series of movement commands (e.g., forward, left, right, backward) and use loops to process these commands, updating the rover\'s position accordingly.\\n\\n**Requirements:**\\n\\n1. The program should start by initializing the rover\'s position at the origin (0, 0) on the grid.\\n2. The user shou

# Intializing a question json

In [27]:
question_json = {}

# Getting problem statement

In [58]:
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage, AIMessage


model = ChatGroq(model="llama-3.1-70b-versatile", temperature=1, max_tokens=5000)

theme = "ipl"
topic = "loops"

messages_1 = [
    SystemMessage(content="Give a name for a python programming question based on the theme and the topic. JUST OUTPUT THE TITLE AND NOTHING ELSE"),
    HumanMessage(content=f"topic : cricket, theme : functions"),
    AIMessage(content="Scores of the batsman in his two innings"),
    HumanMessage(content=f"topic : {topic}, theme : {theme}")
]


prompt_1 = ChatPromptTemplate.from_messages(messages_1)



chain_1 = prompt_1 | model | StrOutputParser()

problem_name = chain_1.invoke({})

problem_name


'"Tight Finish: Analyzing Team Run Totals for Every IPL Season"'

# Getting Description

In [59]:

messages_2 = [
    SystemMessage(content="Give a proper 5 to 10 line paragraph description for the problem title. JUST GIVE THE PROBLEM DESCRIPTION WITHOUT ANY OTHER MESSAGES"),
    HumanMessage(content=f"topic : cricket, theme : functions, problem_statement : Scores of the batsman in his two innings"),
    AIMessage(content="""Write a Python program that defines a function to calculate and display the total and average score of a batsman in two innings. The function should take the scores of the two innings as parameters.

    Define a function named calculate_scores that takes two parameters: score_innings1 and score_innings2.
    The function should print the total and average scores.
    
    Example Output:
    Enter the score of the batsman in the first innings: 45
    Enter the score of the batsman in the second innings: 67
    Total score of the batsman: 112
    Average score of the batsman: 56.0

    Hints:
    Use the input() function to get user input.
    Use int() to convert the input from string to an integer.
                            """),
    HumanMessage(content=f"topic : {topic}, theme : {theme}, problem_statement : {problem_name}")
]


prompt_2 = ChatPromptTemplate.from_messages(messages_2)



chain_2 = prompt_2 | model | StrOutputParser()

description = chain_2.invoke({})

print(description)


Write a Python program to calculate and display the total runs scored by a team in each IPL season. The program should use a loop to iterate over the runs scored in each season and calculate the total runs scored. The program should also display the season with the highest total runs scored.

Assume that the runs scored by the team in each IPL season from 2008 to 2020 are as follows:

2008: 1000, 2009: 1100, 2010: 1200, 2011: 1300, 2012: 1400, 2013: 1500, 2014: 1600, 2015: 1700, 2016: 1800, 2017: 1900, 2018: 2000, 2019: 2100, 2020: 2200

Use a for loop to iterate over the seasons and calculate the total runs scored. Use the max() function to find the season with the highest total runs scored. Display the total runs scored in each season and the season with the highest total runs scored.


# Getting function name

In [60]:

messages_3 = [
    SystemMessage(content="Give a function name and parameters for the following problem statement and description. JUST OUTPUT A SINGLE JSON WITHOUT ANY OTHER SENTANCES"),
    HumanMessage(content=f"""topic : cricket, theme : functions, problem_statement : Scores of the batsman in his two innings, description : A batsman has scored runs in two innings of a cricket match. Write a Python program that:

                            1. Prompts the user to enter the scores of the batsman in the first and second innings.
                            2. Calculates the total score by adding the scores of both innings.
                            3. Calculates the average score by dividing the total score by 2.
                            4. Displays the total and average scores.

                            Example Output:
                            Enter the score of the batsman in the first innings: 45
                            Enter the score of the batsman in the second innings: 67
                            Total score of the batsman: 112
                            Average score of the batsman: 56.0
                            
                            Hint:

                            You can use the input() function to get user input.
                            Use int() to convert the input from string to an integer.
                            Use basic arithmetic operations to calculate the total and average scores."""),
    AIMessage(content="{\"function_name\" : \"compute_total_score\", \"parameters\" : [\"score1\", \"score2\"]}"),
    HumanMessage(content=f"topic : {topic}, theme : {theme}, problem_statement : {problem_statement}, description : {description}")
]


prompt_3 = ChatPromptTemplate.from_messages(messages_3)



chain_3 = prompt_3 | model | JsonOutputParser()

function_json = chain_3.invoke({})

print(function_json)


{'function_name': 'calculate_total_runs', 'parameters': ['runs_per_season']}


# Get function template

In [61]:

messages_4 = [
    SystemMessage(content="Give a function name and parameters, give the function template. JUST GIVE THE TEMPLATE AND NO OTHER SENTANCES"),
    HumanMessage(content=f"function_name : compute_total_score, parameters : [score1, score2]"),
    AIMessage(content="def compute_total_score(score1, score2):\\n # Write your code here\\n return"),
    HumanMessage(content=f"function_name : {function_json['function_name']}, parameters : {function_json['parameters']}")
]


prompt_4 = ChatPromptTemplate.from_messages(messages_4)



chain_4 = prompt_4 | model | StrOutputParser()

function_template = chain_4.invoke({})

print(function_template)

def calculate_total_runs(runs_per_season):\n # Write your code here\n return


# Get Input Outputs for the problem

In [62]:

messages_5 = [
    SystemMessage(content="Give only explainaton for one of the input output sample. It should be just a small description in 5 to 10 lines"),
    HumanMessage(content=f"""function_name : compute_total_score, parameters : [score1, score2], description :
    Write a Python program that defines a function to calculate and display the total and average score of a batsman in two innings. The function should take the scores of the two innings as parameters.

    Define a function named calculate_scores that takes two parameters: score_innings1 and score_innings2.
    The function should print the total and average scores.

    Example Output:
    Enter the score of the batsman in the first innings: 45
    Enter the score of the batsman in the second innings: 67
    Total score of the batsman: 112
    Average score of the batsman: 56.0

    Hints:
    Use the input() function to get user input.
    Use int() to convert the input from string to an integer."""),
    AIMessage(content="""
Input :
Enter the score of the batsman in the first innings: 45
Enter the score of the batsman in the second innings: 67
Output:
Total score of the batsman: 112
Average score of the batsman: 56.0

Explanation:

The program first prompts the user to enter the score for the first innings. In this case, the user enters 45.
Then, it prompts for the score of the second innings. The user enters 67.
The calculate_scores function takes these two scores as parameters.
Inside the function:

It calculates the total score by adding the two innings scores: 45 + 67 = 112
It calculates the average score by dividing the total by 2 (since there are two innings): 112 / 2 = 56.0


The function then prints the results:

The total score (112)
The average score (56.0)



The average is displayed with a decimal point to show that it's a floating-point number, which allows for more precise averages in cases where the total isn't evenly divisible by 2."""),
    HumanMessage(content=f"function_name : {function_json['function_name']}, parameters : {function_json['parameters']}, description : {description}")
]


prompt_5 = ChatPromptTemplate.from_messages(messages_5)



chain_5 = prompt_5 | model | StrOutputParser()

input_output_description = chain_5.invoke({})

print(input_output_description)


Input :
{'2008': 1000, '2009': 1100, '2010': 1200, '2011': 1300, '2012': 1400, '2013': 1500, '2014': 1600, '2015': 1700, '2016': 1800, '2017': 1900, '2018': 2000, '2019': 2100, '2020': 2200}

Explanation:

The program uses a for loop to iterate over the seasons and their corresponding runs scored. 

Inside the loop:

It calculates the total runs scored in each season by adding the runs scored in that season to a running total.

It keeps track of the season with the highest total runs scored by comparing the current season's runs to the maximum runs scored so far.

After the loop finishes iterating over all the seasons:

It displays the total runs scored in each season.

It displays the season with the highest total runs scored.


In [63]:
messages_6 = [
    SystemMessage(content="Give a minimum of 4 INPUT OUTPUT Pairs for the given problem statement and description. OUTPUT ONLY JSON PARSABLE OBJECT WITHOUT ANY OTHER STATEMENTS"),
    HumanMessage(content=f"""problem_statement : Scores of the batsman in his two innings, description : Write a Python function that calculates the total score of a batsman across two innings. The function should take two arguments representing the scores of the batsman in the two innings and return the total score
    input_output_description :
Input :
Enter the score of the batsman in the first innings: 45
Enter the score of the batsman in the second innings: 67
Output:
Total score of the batsman: 112
Average score of the batsman: 56.0

Explanation:

The program first prompts the user to enter the score for the first innings. In this case, the user enters 45.
Then, it prompts for the score of the second innings. The user enters 67.
The calculate_scores function takes these two scores as parameters.
Inside the function:

It calculates the total score by adding the two innings scores: 45 + 67 = 112
It calculates the average score by dividing the total by 2 (since there are two innings): 112 / 2 = 56.0


The function then prints the results:

The total score (112)
The average score (56.0)



The average is displayed with a decimal point to show that it's a floating-point number, which allows for more precise averages in cases where the total isn't evenly divisible by 2."""),
    AIMessage(content="[{\"input\" : [10, 20], \"output\" : 30}, {\"input\" : [15, 89], \"output\" : 104}]"),
    HumanMessage(content=f"problem_statement : {problem_statement}, description : {description}, input_output_description : {input_output_description}")
]

prompt_6 = ChatPromptTemplate.from_messages(messages_6)

chain_6 = prompt_6 | model | JsonOutputParser()

input_output = chain_6.invoke({})

print(input_output)


[{'input': {'2008': 1000, '2009': 1100, '2010': 1200, '2011': 1300, '2012': 1400, '2013': 1500, '2014': 1600, '2015': 1700, '2016': 1800, '2017': 1900, '2018': 2000, '2019': 2100, '2020': 2200}, 'output': 'Season: 2020, Total Runs: 2200, Season with Highest Runs: 2020'}, {'input': {'2010': 1500, '2011': 1600, '2012': 1700, '2013': 1800, '2014': 1900, '2015': 2000, '2016': 2100, '2017': 2200, '2018': 2300, '2019': 2400, '2020': 2500}, 'output': 'Season: 2020, Total Runs: 2500, Season with Highest Runs: 2020'}, {'input': {'2005': 1200, '2006': 1300, '2007': 1400, '2008': 1500, '2009': 1600, '2010': 1700, '2011': 1800, '2012': 1900, '2013': 2000, '2014': 2100, '2015': 2200, '2016': 2300, '2017': 2400, '2018': 2500, '2019': 2600, '2020': 2700}, 'output': 'Season: 2020, Total Runs: 2700, Season with Highest Runs: 2020'}, {'input': {'2012': 1400, '2013': 1500, '2014': 1600, '2015': 1700, '2016': 1800, '2017': 1900, '2018': 2000, '2019': 2100, '2020': 2200, '2021': 2300, '2022': 2400, '2023':

In [65]:
question = {
    "problem_name": problem_name,
    "description": description,
    "function_info": function_json,
    "function_template": function_template,
    "input_output_description": input_output_description,
    "input_output_pairs": input_output
}

# Convert the final result to JSON
import json
question = json.dumps(question, indent=2)

print(question)

{
  "problem_name": "\"Tight Finish: Analyzing Team Run Totals for Every IPL Season\"",
  "description": "Write a Python program to calculate and display the total runs scored by a team in each IPL season. The program should use a loop to iterate over the runs scored in each season and calculate the total runs scored. The program should also display the season with the highest total runs scored.\n\nAssume that the runs scored by the team in each IPL season from 2008 to 2020 are as follows:\n\n2008: 1000, 2009: 1100, 2010: 1200, 2011: 1300, 2012: 1400, 2013: 1500, 2014: 1600, 2015: 1700, 2016: 1800, 2017: 1900, 2018: 2000, 2019: 2100, 2020: 2200\n\nUse a for loop to iterate over the seasons and calculate the total runs scored. Use the max() function to find the season with the highest total runs scored. Display the total runs scored in each season and the season with the highest total runs scored.",
  "function_info": {
    "function_name": "calculate_total_runs",
    "parameters": [
  

# Making a whole class

In [73]:
import json
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain_groq import ChatGroq

class ProblemGenerator:
    def __init__(self, model_name="llama-3.1-70b-versatile", temperature=1, max_tokens=5000):
        self.model = ChatGroq(model=model_name, temperature=temperature, max_tokens=max_tokens)
        self.theme = ""
        self.topic = ""
        self.result = {}

    def set_theme_and_topic(self, theme, topic):
        self.theme = theme
        self.topic = topic

    def generate_problem_name(self):
        messages = [
            SystemMessage(content="Give a name for a python programming question based on the theme and the topic. JUST OUTPUT THE TITLE AND NOTHING ELSE"),
            HumanMessage(content=f"topic : cricket, theme : functions"),
            AIMessage(content="Scores of the batsman in his two innings"),
            HumanMessage(content=f"topic : {self.topic}, theme : {self.theme}")
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        chain = prompt | self.model | StrOutputParser()
        self.result['problem_name'] = chain.invoke({})

    def generate_description(self):
        messages = [
            SystemMessage(content="Give a proper 5 to 10 line paragraph description for the problem title. JUST GIVE THE PROBLEM DESCRIPTION WITHOUT ANY OTHER MESSAGES"),
            HumanMessage(content=f"topic : cricket, theme : functions, problem_statement : Scores of the batsman in his two innings"),
            AIMessage(content="""Write a Python program that defines a function to calculate and display the total and average score of a batsman in two innings. The function should take the scores of the two innings as parameters.
            Define a function named calculate_scores that takes two parameters: score_innings1 and score_innings2.
            The function should print the total and average scores.
            
            Example Output:
            Enter the score of the batsman in the first innings: 45
            Enter the score of the batsman in the second innings: 67
            Total score of the batsman: 112
            Average score of the batsman: 56.0
            Hints:
            Use the input() function to get user input.
            Use int() to convert the input from string to an integer.
                                    """),
            HumanMessage(content=f"topic : {self.topic}, theme : {self.theme}, problem_statement : {self.result['problem_name']}")
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        chain = prompt | self.model | StrOutputParser()
        self.result['description'] = chain.invoke({})

    def generate_function_info(self):
        messages = [
            SystemMessage(content="Give a function name and parameters for the following problem statement and description. JUST OUTPUT A SINGLE JSON WITHOUT ANY OTHER SENTANCES"),
            HumanMessage(content=f"""topic : cricket, theme : functions, problem_statement : Scores of the batsman in his two innings, description : A batsman has scored runs in two innings of a cricket match. Write a Python program that:
                                    1. Prompts the user to enter the scores of the batsman in the first and second innings.
                                    2. Calculates the total score by adding the scores of both innings.
                                    3. Calculates the average score by dividing the total score by 2.
                                    4. Displays the total and average scores.
                                    Example Output:
                                    Enter the score of the batsman in the first innings: 45
                                    Enter the score of the batsman in the second innings: 67
                                    Total score of the batsman: 112
                                    Average score of the batsman: 56.0
                                    
                                    Hint:
                                    You can use the input() function to get user input.
                                    Use int() to convert the input from string to an integer.
                                    Use basic arithmetic operations to calculate the total and average scores."""),
            AIMessage(content='{"function_name" : "compute_total_score", "parameters" : ["score1", "score2"]}'),
            HumanMessage(content=f"topic : {self.topic}, theme : {self.theme}, problem_statement : {self.result['problem_name']}, description : {self.result['description']}")
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        chain = prompt | self.model | JsonOutputParser()
        self.result['function_info'] = chain.invoke({})

    def generate_function_template(self):
        messages = [
            SystemMessage(content="Give a function name and parameters, give the function template. JUST GIVE THE TEMPLATE AND NO OTHER SENTANCES"),
            HumanMessage(content=f"function_name : compute_total_score, parameters : [score1, score2]"),
            AIMessage(content="def compute_total_score(score1, score2):\\n # Write your code here\\n return"),
            HumanMessage(content=f"function_name : {self.result['function_info']['function_name']}, parameters : {self.result['function_info']['parameters']}")
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        chain = prompt | self.model | StrOutputParser()
        self.result['function_template'] = chain.invoke({})

    def generate_input_output_description(self):
        messages = [
            SystemMessage(content="Give only explainaton for one of the input output sample. It should be just a small description in 5 to 10 lines. NO CODE TO BE PROVIDED."),
            HumanMessage(content=f"""function_name : compute_total_score, parameters : [score1, score2], description :
            Write a Python program that defines a function to calculate and display the total and average score of a batsman in two innings. The function should take the scores of the two innings as parameters.
            Define a function named calculate_scores that takes two parameters: score_innings1 and score_innings2.
            The function should print the total and average scores.
            Example Output:
            Enter the score of the batsman in the first innings: 45
            Enter the score of the batsman in the second innings: 67
            Total score of the batsman: 112
            Average score of the batsman: 56.0
            Hints:
            Use the input() function to get user input.
            Use int() to convert the input from string to an integer."""),
            AIMessage(content="""
        Input :
        Enter the score of the batsman in the first innings: 45
        Enter the score of the batsman in the second innings: 67
        Output:
        Total score of the batsman: 112
        Average score of the batsman: 56.0
        Explanation:
        The program first prompts the user to enter the score for the first innings. In this case, the user enters 45.
        Then, it prompts for the score of the second innings. The user enters 67.
        The calculate_scores function takes these two scores as parameters.
        Inside the function:
        It calculates the total score by adding the two innings scores: 45 + 67 = 112
        It calculates the average score by dividing the total by 2 (since there are two innings): 112 / 2 = 56.0
        The function then prints the results:
        The total score (112)
        The average score (56.0)
        The average is displayed with a decimal point to show that it's a floating-point number, which allows for more precise averages in cases where the total isn't evenly divisible by 2."""),
            HumanMessage(content=f"function_name : {self.result['function_info']['function_name']}, parameters : {self.result['function_info']['parameters']}, description : {self.result['description']}")
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        chain = prompt | self.model | StrOutputParser()
        self.result['input_output_description'] = chain.invoke({})

    def generate_input_output_pairs(self):
        messages = [
            SystemMessage(content="Give a minimum of 4 INPUT OUTPUT Pairs for the given problem statement and description. OUTPUT ONLY JSON PARSABLE OBJECT WITHOUT ANY OTHER STATEMENTS"),
            HumanMessage(content=f"""problem_statement : Scores of the batsman in his two innings, description : Write a Python function that calculates the total score of a batsman across two innings. The function should take two arguments representing the scores of the batsman in the two innings and return the total score
            input_output_description :
        Input :
        Enter the score of the batsman in the first innings: 45
        Enter the score of the batsman in the second innings: 67
        Output:
        Total score of the batsman: 112
        Average score of the batsman: 56.0
        Explanation:
        The program first prompts the user to enter the score for the first innings. In this case, the user enters 45.
        Then, it prompts for the score of the second innings. The user enters 67.
        The calculate_scores function takes these two scores as parameters.
        Inside the function:
        It calculates the total score by adding the two innings scores: 45 + 67 = 112
        It calculates the average score by dividing the total by 2 (since there are two innings): 112 / 2 = 56.0
        The function then prints the results:
        The total score (112)
        The average score (56.0)
        The average is displayed with a decimal point to show that it's a floating-point number, which allows for more precise averages in cases where the total isn't evenly divisible by 2."""),
            AIMessage(content='[{"input" : [10, 20], "output" : 30}, {"input" : [15, 89], "output" : 104}]'),
            HumanMessage(content=f"problem_statement : {self.result['problem_name']}, description : {self.result['description']}, input_output_description : {self.result['input_output_description']}")
        ]
        prompt = ChatPromptTemplate.from_messages(messages)
        chain = prompt | self.model | JsonOutputParser()
        self.result['input_output_pairs'] = chain.invoke({})

    def generate_problem(self):
        self.generate_problem_name()
        self.generate_description()
        self.generate_function_info()
        self.generate_function_template()
        self.generate_input_output_description()
        self.generate_input_output_pairs()

    def get_json_result(self):
        return json.dumps(self.result, indent=2)



In [74]:
generator = ProblemGenerator()
generator.set_theme_and_topic("ipl", "loops")
generator.generate_problem()
print(generator.get_json_result())

{
  "problem_name": "Iterating Over IPL Team Rosters",
  "description": "Write a Python program that uses loops to iterate over the rosters of two IPL teams, Mumbai Indians and Chennai Super Kings. The program should print the names of all players in both teams. The roster of each team is stored in a list, where each element is a dictionary containing the player's name and role (batsman, bowler, or all-rounder). The program should use nested loops to iterate over the teams and their respective rosters. \n\nExample Output:\nMumbai Indians:\nRohit Sharma - Batsman\nHardik Pandya - All-rounder\nJasprit Bumrah - Bowler\n...\nChennai Super Kings:\nMS Dhoni - Batsman\nRavindra Jadeja - All-rounder\nShane Watson - Batsman\n...\n\nTeam Rosters:\nMumbai Indians = [\n    {'name': 'Rohit Sharma', 'role': 'Batsman'},\n    {'name': 'Hardik Pandya', 'role': 'All-rounder'},\n    {'name': 'Jasprit Bumrah', 'role': 'Bowler'},\n    ...\n]\n\nChennai Super Kings = [\n    {'name': 'MS Dhoni', 'role': 'Bat